In [1]:
import torch
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import wandb
from datasets import load_from_disk
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import roc_auc_score, confusion_matrix, multilabel_confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import classification_report, plot_roc_curve
from argparse import Namespace

In [2]:
config = {
    "train_subset": 1500000,
    "valid_subset": 400000,
    "test_subset" : 200000,
    "seed": 42,
    "wandb_project_name": "rf_param_opt",
    "count": 20   # Number of runs for the Sweep
}

args = Namespace(**config)

### Define the Sweep config

In [3]:
sweep_config = {
    'method' : 'random',
    'metric': {
        'name': 'roc_auc',
        'goal': 'maximize'
    },
    'parameters': {
        'criterion': {'values': ['gini', 'entropy']},
        'n_estimators': {'values': [50, 500, 1000]},
        'min_samples_split': {'values': [10, 100, 1000]},
        'min_samples_leaf': {'values': [10, 50, 100]},
    }
}

### Log in to Weight and Biases

In [4]:
wandb.login()

wandb: Currently logged in as: m2im (npsdaor). Use `wandb login --relogin` to force relogin


True

### Read the dataset

In [5]:
violence_hidden = load_from_disk("../../Violence_data/geo_corpus.0.0.1_datasets_hidden_xlmt")

In [6]:
violence_hidden

DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask', 'hidden_state'],
        num_rows: 16769932
    })
    validation: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask', 'hidden_state'],
        num_rows: 4192483
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask', 'hidden_state'],
        num_rows: 2329158
    })
})

In [7]:
# Remove unncesary columns
keep_cols = ['hidden_state', 'labels']
remove_columns = [col for col in violence_hidden['train'].column_names if col not in keep_cols]

In [8]:
violence_hidden = violence_hidden.remove_columns(remove_columns)

In [9]:
violence_hidden

DatasetDict({
    train: Dataset({
        features: ['labels', 'hidden_state'],
        num_rows: 16769932
    })
    validation: Dataset({
        features: ['labels', 'hidden_state'],
        num_rows: 4192483
    })
    test: Dataset({
        features: ['labels', 'hidden_state'],
        num_rows: 2329158
    })
})

In [10]:
# Extract a subset of the dataset
train_clf_ds = violence_hidden["train"].shuffle(args.seed).select(range(args.train_subset))
validation_clf_ds = violence_hidden["validation"].shuffle(args.seed).select(range(args.valid_subset))
test_clf_ds = violence_hidden["test"].shuffle(args.seed).select(range(args.test_subset))

Loading cached shuffled indices for dataset at ../../Violence_data/geo_corpus.0.0.1_datasets_hidden_xlmt/train/cache-333532fe06490884.arrow
Loading cached shuffled indices for dataset at ../../Violence_data/geo_corpus.0.0.1_datasets_hidden_xlmt/validation/cache-879cab607103254e.arrow
Loading cached shuffled indices for dataset at ../../Violence_data/geo_corpus.0.0.1_datasets_hidden_xlmt/test/cache-9d513c2ddd67e482.arrow


In [ ]:
train_clf_ds[0]

### Create a feature matrix

In [11]:
%time X_train = np.array(train_clf_ds["hidden_state"])
y_train = np.array(train_clf_ds["labels"])
X_validation = np.array(validation_clf_ds["hidden_state"])
y_validation = np.array(validation_clf_ds["labels"])
X_test = np.array(test_clf_ds["hidden_state"])
y_test = np.array(test_clf_ds["labels"])
X_train.shape

CPU times: user 24 s, sys: 20.6 s, total: 44.6 s
Wall time: 29.9 s


(1500000, 768)

## Helper Functions

In [12]:
# Create a function to report the various metrics for each classifier
def metricsReport(test_labels, predictions):
    
    roc_auc = roc_auc_score(test_labels, predictions, average = "micro")

    weighted_precision = precision_score(test_labels, predictions, average='weighted')
    weighted_recall = recall_score(test_labels, predictions, average='weighted')
    weighted_f1 = f1_score(test_labels, predictions, average='weighted')

    micro_precision = precision_score(test_labels, predictions, average='micro')
    micro_recall = recall_score(test_labels, predictions, average='micro')
    micro_f1 = f1_score(test_labels, predictions, average='micro')
    
    
    return {"roc_auc": format(roc_auc, '.4f'), "weighted_precision": format(weighted_precision, '.4f'),
                                    "weighted_recall": format(weighted_recall, '.4f'), "weighted_f1": format(weighted_f1, '.4f'), "micro_precision": format(micro_precision, '.4f'),
                                    "micro_recall": format(micro_recall, '.4f'), "micro_f1": format(micro_f1, '.4f')}

In [13]:
def plot_confusion_matrix(y_test, y_pred, clf:str):
    f, axes = plt.subplots(2, 3, figsize=(25, 15))
    f.suptitle(clf, fontsize=36)
    axes = axes.ravel()
    for i in range(6):
        labels=['post7geo10', 'post7geo30', 'post7geo50','pre7geo10','pre7geo30', 'pre7geo50']
        disp = ConfusionMatrixDisplay(confusion_matrix(y_test[:, i],
                                                       y_pred[:, i]),
                                      display_labels=[0, i])
        disp.plot(ax=axes[i], values_format='.4g')
        disp.ax_.set_title(labels[i])
        if i<10:
            disp.ax_.set_xlabel('')
        if i%5!=0:
            disp.ax_.set_ylabel('')
        disp.im_.colorbar.remove()

    plt.subplots_adjust(wspace=0.10, hspace=0.1)
    f.colorbar(disp.im_, ax=axes)
    plt.show()

# Random Forest Classifier

In [14]:
def train():
    with wandb.init():
        config = wandb.config
        rfClassifier = RandomForestClassifier(
            n_jobs=-1, 
            random_state=args.seed,
            n_estimators = config.n_estimators,
            min_samples_split = config.min_samples_split,
            min_samples_leaf = config.min_samples_leaf)
        rfClassifier.fit(X_train, y_train)
        rfPreds = rfClassifier.predict(X_test)
        scores = metricsReport(y_test, rfPreds)
        roc_auc = float(scores["roc_auc"])
        prec_w = float(scores["weighted_precision"])
        recall_w = float(scores["weighted_recall"])
        f1_w = float(scores["weighted_f1"])
        prec_micro = float(scores["micro_precision"])
        recall_micro = float(scores["micro_recall"])
        f1_micro = float(scores["micro_f1"])
    
        wandb.log({'roc_auc': roc_auc,
              'weighted_precision': prec_w,
              'weighted_recall': recall_w,
              'weighted_f1': f1_w,
              'micro_precision': prec_micro,
              'micro_recall': recall_micro,
              'micro_f1': f1_micro})

### Initialize the Sweep

In [15]:
sweep_id = wandb.sweep(sweep=sweep_config, project=args.wandb_project_name)

Create sweep with ID: e9ww50tw
Sweep URL: https://wandb.ai/npsdaor/rf_param_opt/sweeps/e9ww50tw


### Start the Sweep agent

In [16]:
wandb.agent(sweep_id, function=train, count=args.count)

wandb: Agent Starting Run: 8yo4teer with config:
wandb: 	criterion: entropy
wandb: 	min_samples_leaf: 50
wandb: 	min_samples_split: 100
wandb: 	n_estimators: 50


micro_f1,▁
micro_precision,▁
micro_recall,▁
roc_auc,▁
weighted_f1,▁
weighted_precision,▁
weighted_recall,▁
micro_f1,0.5505
micro_precision,0.579
micro_recall,0.5246
roc_auc,0.6003


wandb: Agent Starting Run: gde3904h with config:
wandb: 	criterion: entropy
wandb: 	min_samples_leaf: 10
wandb: 	min_samples_split: 10
wandb: 	n_estimators: 50


micro_f1,▁
micro_precision,▁
micro_recall,▁
roc_auc,▁
weighted_f1,▁
weighted_precision,▁
weighted_recall,▁
micro_f1,0.5537
micro_precision,0.5805
micro_recall,0.5293
roc_auc,0.6022


wandb: Agent Starting Run: dmmawwfd with config:
wandb: 	criterion: gini
wandb: 	min_samples_leaf: 50
wandb: 	min_samples_split: 1000
wandb: 	n_estimators: 1000


...

micro_f1,▁
micro_precision,▁
micro_recall,▁
roc_auc,▁
weighted_f1,▁
weighted_precision,▁
weighted_recall,▁
micro_f1,0.5372
micro_precision,0.5773
micro_recall,0.5024
roc_auc,0.595


wandb: Agent Starting Run: mkc3kpzu with config:
wandb: 	criterion: entropy
wandb: 	min_samples_leaf: 10
wandb: 	min_samples_split: 10
wandb: 	n_estimators: 50


micro_f1,▁
micro_precision,▁
micro_recall,▁
roc_auc,▁
weighted_f1,▁
weighted_precision,▁
weighted_recall,▁
micro_f1,0.5537
micro_precision,0.5805
micro_recall,0.5293
roc_auc,0.6022


wandb: Agent Starting Run: x3agtxbf with config:
wandb: 	criterion: gini
wandb: 	min_samples_leaf: 50
wandb: 	min_samples_split: 100
wandb: 	n_estimators: 50


micro_f1,▁
micro_precision,▁
micro_recall,▁
roc_auc,▁
weighted_f1,▁
weighted_precision,▁
weighted_recall,▁
micro_f1,0.5505
micro_precision,0.579
micro_recall,0.5246
roc_auc,0.6003
